<a href="https://colab.research.google.com/github/Babitha0902/Sentiment_Analysis-intel_unnati-/blob/main/Sentiment_Analysis_IU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
!pip install transformers pandas scikit-learn openvino-dev[onnx] gradio plotly numpy tqdm --upgrade -q
print("Libraries installed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.7/848.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv('/content/Reviews.csv', encoding='latin1', on_bad_lines='skip')
print(f"Dataset loaded with {len(df)} rows!")

# Use the full dataset (no sampling) for maximum accuracy
# Enhanced text cleaning function
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^A-Za-z0-9\s.,!?]', '', text)  # Keep some punctuation for context
    text = re.sub(r'\s+', ' ', text)  # Normalize spaces
    return text.strip() or "no content"

df['Text'] = df['Text'].apply(clean_text)
df = df[df['Text'] != "no content"]  # Remove empty reviews

# Binary sentiment: 1 (positive, score >= 4), 0 (negative, score <= 2)
df['Sentiment'] = df['Score'].apply(lambda x: 1 if x >= 4 else 0 if x <= 2 else -1)
df = df[df['Sentiment'] != -1]  # Drop neutral reviews

# Balance dataset
df_positive = df[df['Sentiment'] == 1].sample(n=min(len(df[df['Sentiment'] == 0]), len(df[df['Sentiment'] == 1])), random_state=42)
df_negative = df[df['Sentiment'] == 0]
df = pd.concat([df_positive, df_negative])
print(f"Balanced dataset size: {len(df)}")

# Split into train, validation, and test sets (80-10-10)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df['Text'].tolist(), df['Sentiment'].tolist(), test_size=0.2, random_state=42
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42
)
print(f"Training samples: {len(train_texts)}, Validation samples: {len(val_texts)}, Test samples: {len(test_texts)}")

Dataset loaded with 6263 rows!
Balanced dataset size: 1848
Training samples: 1478, Validation samples: 185, Test samples: 185


In [ ]:
from transformers import BertTokenizer
import torch

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}, GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

# Use BERT (larger model) instead of DistilBERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize with a higher max length for better context
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=256)
print("Tokenization complete!")

# Create PyTorch datasets
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)
test_dataset = ReviewDataset(test_encodings, test_labels)
print("Datasets created!")

Using device: cuda, GPU: Tesla T4


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Tokenization complete!
Datasets created!


In [ ]:
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
import numpy as np

# Load BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)

# Data loaders with adaptive batch size
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)  # Larger batch size if memory allows
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Optimizer and scheduler with fine-tuned parameters
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)  # Add weight decay for regularization
total_steps = len(train_loader) * 10  # 10 epochs for deeper training
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=total_steps)

# Training loop with early stopping
num_epochs = 50  # Allow extensive training
best_accuracy = 0
patience = 5
no_improve = 0

for epoch in range(num_epochs):
    model.train()
    print(f"Epoch {epoch + 1}/{num_epochs}")
    progress_bar = tqdm(train_loader, desc="Training")
    total_loss = 0
    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        progress_bar.set_postfix({'loss': loss.item()})

    avg_train_loss = total_loss / len(train_loader)
    print(f"Average Training Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=-1)
            correct += (predictions == batch['labels']).sum().item()
            total += batch['labels'].size(0)
    accuracy = correct / total
    print(f"Validation Accuracy: {accuracy:.4f}")

    # Early stopping and model saving
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        model.save_pretrained("best_model")
        tokenizer.save_pretrained("best_model")
        print("Best model saved!")
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= patience:
            print("Early stopping triggered!")
            break

# Evaluate on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        correct += (predictions == batch['labels']).sum().item()
        total += batch['labels'].size(0)
test_accuracy = correct / total
print(f"Final Test Accuracy: {test_accuracy:.4f}")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/50


Training:   0%|          | 0/93 [00:00<?, ?it/s]

Average Training Loss: 0.6908
Validation Accuracy: 0.7135
Best model saved!
Epoch 2/50


...

Average Training Loss: 0.0051
Validation Accuracy: 0.9189
Epoch 14/50


Training:   0%|          | 0/93 [00:00<?, ?it/s]

Average Training Loss: 0.0008
Validation Accuracy: 0.9189
Epoch 15/50


Training:   0%|          | 0/93 [00:00<?, ?it/s]

Average Training Loss: 0.0023
Validation Accuracy: 0.9189
Early stopping triggered!
Final Test Accuracy: 0.9297


In [ ]:
!pip install openvino-dev[onnx]

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nx-cugraph-cu12 25.2.0 requires networkx>=3.2, but you have networkx 3.1 which is incompatible.


In [ ]:
from openvino.runtime import Core
from transformers import AutoModelForSequenceClassification

# Load the trained model
model = AutoModelForSequenceClassification.from_pretrained("best_model")
tokenizer = BertTokenizer.from_pretrained("best_model")

# Export to ONNX
dummy_input = tokenizer("This is a test", return_tensors="pt", padding=True, truncation=True, max_length=256)
torch.onnx.export(
    model,
    (dummy_input['input_ids'], dummy_input['attention_mask']),
    "sentiment_model.onnx",
    input_names=['input_ids', 'attention_mask'],
    output_names=['logits'],
    dynamic_axes={'input_ids': {0: 'batch_size'}, 'attention_mask': {0: 'batch_size'}, 'logits': {0: 'batch_size'}}
)

# Convert ONNX to OpenVINO IR
!mo --input_model sentiment_model.onnx --output_dir ./openvino_model --data_type FP16
print("Model converted to OpenVINO IR format!")

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
usage: main.py [options]
main.py: error: unrecognized arguments: --data_type FP16
Model converted to OpenVINO IR format!


In [ ]:
!pip install gradio plotly numpy tqdm --upgrade -q

In [ ]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import plotly.graph_objects as go
import numpy as np
from tqdm import tqdm
import pandas as pd
import plotly.io as pio

# Ensure Plotly renders correctly in the environment
pio.renderers.default = "iframe"  # Use iframe renderer for better compatibility in Colab/Jupyter

# Load the best model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained("best_model").to(device)
tokenizer = AutoTokenizer.from_pretrained("best_model")
model.eval()
print(f"Model loaded on {device}")

# Text cleaning function (from Step 2)
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^A-Za-z0-9\s.,!?]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip() or "no content"

# Batch processing function
def process_batch(texts, batch_size=32):
    sentiments = []
    confidences = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Processing"):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        for prob in probs:
            sentiment = "Positive" if prob[1] > prob[0] else "Negative"
            confidence = max(prob)
            sentiments.append(sentiment)
            confidences.append(confidence)
    return sentiments, confidences

# Function to generate recommendations based on metrics
def generate_recommendations(metrics):
    """Generate actionable recommendations based on sentiment metrics"""
    recommendations = []

    if metrics['positive_pct'] > 70 and metrics['avg_confidence'] > 0.7:
        recommendations.append("✅ Strong positive sentiment detected - consider promoting these aspects")
    elif metrics['positive_pct'] < 30 and metrics['avg_confidence'] > 0.7:
        recommendations.append("⚠️ Strong negative sentiment detected - urgent attention needed")

    if metrics['avg_confidence'] < 0.6:
        recommendations.append("🔍 Low confidence predictions - some reviews may need manual review")

    if not recommendations:
        recommendations.append("➡️ Balanced sentiment - monitor trends over time")

    return "\n".join(recommendations)

# Batch analysis function with graph and recommendations
def accurate_batch_analysis(sample_size, csv_path):
    # Handle CSV path (use default if none uploaded)
    if csv_path is None:
        csv_path = '/content/Reviews.csv'

    try:
        df = pd.read_csv(csv_path, encoding='utf-8', on_bad_lines='skip')
    except Exception as e:
        return None, f"Error loading CSV: {str(e)}"

    df['Text'] = df['Text'].apply(clean_text)
    df = df[df['Text'] != "no content"]
    sample_size = min(int(sample_size), len(df))
    if sample_size <= 0:
        return None, "Error: Sample size must be greater than 0"

    df_sample = df.sample(n=sample_size, random_state=42)
    sentiments, confidences = process_batch(df_sample['Text'].tolist())

    # Calculate counts and metrics
    pos_count = sentiments.count("Positive")
    neg_count = sentiments.count("Negative")
    total = len(sentiments)

    metrics = {
        'positive_pct': (pos_count / total) * 100 if total > 0 else 0,
        'negative_pct': (neg_count / total) * 100 if total > 0 else 0,
        'avg_confidence': np.mean(confidences) if confidences else 0,
        'confidence_std': np.std(confidences) if confidences else 0,
        'total_reviews': total
    }

    # Create a simple horizontal bar graph
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=[pos_count, neg_count],
        y=['Positive', 'Negative'],
        orientation='h',
        marker_color=['#00cc96', '#ef553b'],  # Green for Positive, Red for Negative
        text=[f"{pos_count}", f"{neg_count}"],
        textposition='auto'
    ))
    fig.update_layout(
        title="Sentiment Distribution",
        xaxis_title="Count",
        yaxis_title="Sentiment",
        height=300,
        width=500,
        margin=dict(l=50, r=50, t=50, b=50),
        template="plotly_white"
    )

    # Generate report with recommendations
    report = f"""Analysis Report:
    - Total Reviews: {metrics['total_reviews']}
    - Positive: {metrics['positive_pct']:.1f}%
    - Negative: {metrics['negative_pct']:.1f}%
    - Avg Confidence: {metrics['avg_confidence']:.3f} (±{metrics['confidence_std']:.3f})

    Recommendations:
    {generate_recommendations(metrics)}
    """

    # Debugging: Ensure figure is valid
    print("Figure created successfully")
    return fig, report

# Gradio interface with fallback
with gr.Blocks(title="Sentiment Analysis") as interface:
    gr.Markdown("# Sentiment Analysis")

    with gr.Row():
        with gr.Column():
            sample_size = gr.Slider(
                minimum=100,
                maximum=10000,
                value=1000,
                step=100,
                label="Sample Size"
            )
            csv_upload = gr.File(
                label="Upload CSV (optional)",
                file_types=[".csv"],
                type="filepath"
            )
            analyze_btn = gr.Button("Analyze", variant="primary")

        with gr.Column():
            plot_output = gr.Plot(label="Sentiment Distribution")
            report_output = gr.Textbox(label="Report", lines=10)
            # Fallback: Display graph as HTML if Plot fails
            html_output = gr.HTML(label="Fallback Graph (if Plot fails)")

    # Function to generate HTML fallback
    def create_html_fallback(fig):
        return fig.to_html(include_plotlyjs="cdn")

    # Connect button to function
    analyze_btn.click(
        fn=accurate_batch_analysis,
        inputs=[sample_size, csv_upload],
        outputs=[plot_output, report_output]
    ).then(
        fn=create_html_fallback,
        inputs=plot_output,
        outputs=html_output
    )

# Launch with debug mode
interface.launch(share=True, debug=True)

Model loaded on cuda
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2c311640af5d0952cb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Processing: 100%|██████████| 4/4 [00:01<00:00,  3.12it/s]


Figure created successfully


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2137, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1663, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Figure created successfully


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2137, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1663, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Figure created successfully


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2137, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1663, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Figure created successfully


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2137, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1663, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^